In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import math
import time

########################
# Configuration
########################
ticker = 'UNH'
start_date = '2020-01-01'
end_date = '2024-01-01'
1
# Hyperparameters
n_steps = 2
window_size = 5
step_size = 2
initial_epochs = 100
update_epochs = 100
initial_lr = 0.0005
update_lr = 0.00005
train_ratio = 0.9
n_units = 16
batch_size = 4

# If you change features or other logic, do so below
if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")


def load_full_data_helper(ticker, start_date, end_date, window_size):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None

    # Next day prediction setup
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    # Simple features
    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None
    return X_feat.values, y_raw, dates


def load_full_data(ticker, start_date, end_date, window_size):
    X_feat, y_raw, dates = load_full_data_helper(ticker, start_date, end_date, window_size)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates


def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)

    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)


def create_lstm_model(n_units, learning_rate, n_steps, n_features, n_layers=1):
    inputs = Input(shape=(n_steps, n_features))
    x = LSTM(n_units, activation='relu', return_sequences=(n_layers > 1))(inputs)
    if n_layers > 1:
        x = LSTM(n_units, activation='relu')(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    # We'll compile after loading weights if needed
    return model


def train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                    initial_training=False, prev_weights=None):
    total_samples = len(X_samples)
    global train_ratio
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if X_train.size == 0 or X_val.size == 0:
        return None, np.nan

    n_steps_local = X_train.shape[1]
    n_features = X_train.shape[2]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    model = create_lstm_model(n_units, lr, n_steps_local, n_features, n_layers=1)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr))

    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False,
              validation_data=(X_val, y_val), callbacks=[es])

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape


def rolling_training_early_stopping(ticker, window_size, step_size, n_steps):
    X_full, y_full, dates_full = load_full_data(ticker, start_date, end_date, window_size)
    if X_full is None:
        print("[WARN] Not enough data.")
        return

    start_time = time.time()

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        print("[WARN] Initial window not valid.")
        return
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    print("[INFO] Training initial window with Early Stopping...")
    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=True, prev_weights=None)
    print(f"[RESULT] Initial window validation MAPE: {val_mape:.2f}%")

    weights_path = "rolling_error_analysis_results/cost_initial_es.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            print("[INFO] No more windows can be extracted.")
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        print(f"\n[INFO] Iteration {iteration}: training on new window with Early Stopping...")
        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=weights_path)
        print(f"[RESULT] Window validation MAPE: {val_mape:.2f}%")

        weights_path = f"rolling_error_analysis_results/cost_iteration_{iteration}_es.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])

        test_mape = mean_absolute_percentage_error(y_test_unscaled, y_pred_unscaled)*100
        print(f"[INFO] Predicted last sample of window: Actual={y_test_unscaled[0]:.2f}, Pred={y_pred_unscaled[0]:.2f}, MAPE={test_mape:.2f}%")

        iteration += 1

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    df_results.to_csv("rolling_error_analysis_results/cost_rolling_results_es.csv", index=False)
    overall_mape = df_results['MAPE'].mean()
    print(f"\n[INFO] Rolling training with Early Stopping completed for {ticker}. Overall MAPE: {overall_mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] Total elapsed time: {elapsed:.2f}s")

    worst_10 = df_results.sort_values('MAPE', ascending=False).head(10)
    print("[INFO] Top 10 worst predictions in rolling scenario (Early Stopping):")
    print(worst_10[['Date', 'Actual', 'Predicted', 'MAPE']])

if __name__ == "__main__":
    # Just run with the current top-level defined parameters:
    rolling_training_early_stopping(ticker, window_size, step_size, n_steps)


[*********************100%%**********************]  1 of 1 completed
[INFO] Training initial window with Early Stopping...
[RESULT] Initial window validation MAPE: 0.87%

[INFO] Iteration 1: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.96%
[INFO] Predicted last sample of window: Actual=285.85, Pred=294.32, MAPE=2.96%

[INFO] Iteration 2: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.97%
[INFO] Predicted last sample of window: Actual=296.41, Pred=293.54, MAPE=0.97%

[INFO] Iteration 3: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.63%
[INFO] Predicted last sample of window: Actual=298.47, Pred=296.58, MAPE=0.63%

[INFO] Iteration 4: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.02%
[INFO] Predicted last sample of window: Actual=300.59, Pred=300.52, MAPE=0.02%

[INFO] Iteration 5: training on new window with Early Stopping...
[RESULT] Window validat

[RESULT] Window validation MAPE: 1.02%
[INFO] Predicted last sample of window: Actual=290.96, Pred=287.99, MAPE=1.02%

[INFO] Iteration 45: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.53%
[INFO] Predicted last sample of window: Actual=289.05, Pred=290.59, MAPE=0.53%

[INFO] Iteration 46: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.73%
[INFO] Predicted last sample of window: Actual=286.91, Pred=289.00, MAPE=0.73%

[INFO] Iteration 47: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.02%
[INFO] Predicted last sample of window: Actual=294.89, Pred=288.95, MAPE=2.02%

[INFO] Iteration 48: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 3.18%
[INFO] Predicted last sample of window: Actual=303.97, Pred=294.31, MAPE=3.18%

[INFO] Iteration 49: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.64%
[INFO] Predicted last sample of

[RESULT] Window validation MAPE: 2.04%
[INFO] Predicted last sample of window: Actual=292.14, Pred=298.10, MAPE=2.04%

[INFO] Iteration 90: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 3.01%
[INFO] Predicted last sample of window: Actual=302.50, Pred=293.41, MAPE=3.01%

[INFO] Iteration 91: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.01%
[INFO] Predicted last sample of window: Actual=304.15, Pred=304.13, MAPE=0.01%

[INFO] Iteration 92: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.81%
[INFO] Predicted last sample of window: Actual=313.07, Pred=304.26, MAPE=2.81%

[INFO] Iteration 93: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.93%
[INFO] Predicted last sample of window: Actual=318.15, Pred=312.02, MAPE=1.93%

[INFO] Iteration 94: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.70%
[INFO] Predicted last sample of

[RESULT] Window validation MAPE: 0.77%
[INFO] Predicted last sample of window: Actual=333.58, Pred=336.13, MAPE=0.77%

[INFO] Iteration 134: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.69%
[INFO] Predicted last sample of window: Actual=338.72, Pred=333.00, MAPE=1.69%

[INFO] Iteration 135: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.46%
[INFO] Predicted last sample of window: Actual=329.32, Pred=334.14, MAPE=1.46%

[INFO] Iteration 136: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.68%
[INFO] Predicted last sample of window: Actual=324.34, Pred=326.56, MAPE=0.68%

[INFO] Iteration 137: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.43%
[INFO] Predicted last sample of window: Actual=333.40, Pred=328.62, MAPE=1.43%

[INFO] Iteration 138: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.58%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 1.38%
[INFO] Predicted last sample of window: Actual=400.25, Pred=405.77, MAPE=1.38%

[INFO] Iteration 178: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.20%
[INFO] Predicted last sample of window: Actual=401.10, Pred=400.28, MAPE=0.20%

[INFO] Iteration 179: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.82%
[INFO] Predicted last sample of window: Actual=397.89, Pred=401.14, MAPE=0.82%

[INFO] Iteration 180: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.32%
[INFO] Predicted last sample of window: Actual=400.28, Pred=399.02, MAPE=0.32%

[INFO] Iteration 181: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.16%
[INFO] Predicted last sample of window: Actual=397.35, Pred=397.97, MAPE=0.16%

[INFO] Iteration 182: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.35%
[INFO] Predicted last samp

...

[RESULT] Window validation MAPE: 3.06%
[INFO] Predicted last sample of window: Actual=499.81, Pred=484.51, MAPE=3.06%

[INFO] Iteration 310: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.85%
[INFO] Predicted last sample of window: Actual=505.66, Pred=496.30, MAPE=1.85%

[INFO] Iteration 311: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.84%
[INFO] Predicted last sample of window: Actual=515.71, Pred=506.22, MAPE=1.84%

[INFO] Iteration 312: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.58%
[INFO] Predicted last sample of window: Actual=517.40, Pred=514.39, MAPE=0.58%

[INFO] Iteration 313: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.20%
[INFO] Predicted last sample of window: Actual=515.29, Pred=514.24, MAPE=0.20%

[INFO] Iteration 314: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.56%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 0.00%
[INFO] Predicted last sample of window: Actual=541.80, Pred=541.83, MAPE=0.00%

[INFO] Iteration 354: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.02%
[INFO] Predicted last sample of window: Actual=555.15, Pred=543.95, MAPE=2.02%

[INFO] Iteration 355: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.86%
[INFO] Predicted last sample of window: Actual=543.43, Pred=548.11, MAPE=0.86%

[INFO] Iteration 356: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.66%
[INFO] Predicted last sample of window: Actual=538.17, Pred=541.70, MAPE=0.66%

[INFO] Iteration 357: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.71%
[INFO] Predicted last sample of window: Actual=553.05, Pred=543.57, MAPE=1.71%

[INFO] Iteration 358: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.09%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 1.11%
[INFO] Predicted last sample of window: Actual=473.88, Pred=479.15, MAPE=1.11%

[INFO] Iteration 398: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.18%
[INFO] Predicted last sample of window: Actual=464.91, Pred=470.37, MAPE=1.18%

[INFO] Iteration 399: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.05%
[INFO] Predicted last sample of window: Actual=463.68, Pred=463.92, MAPE=0.05%

[INFO] Iteration 400: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.16%
[INFO] Predicted last sample of window: Actual=465.43, Pred=464.67, MAPE=0.16%

[INFO] Iteration 401: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.02%
[INFO] Predicted last sample of window: Actual=469.50, Pred=469.41, MAPE=0.02%

[INFO] Iteration 402: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.49%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 2.03%
[INFO] Predicted last sample of window: Actual=447.75, Pred=456.86, MAPE=2.03%

[INFO] Iteration 442: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 4.27%
[INFO] Predicted last sample of window: Actual=484.08, Pred=463.43, MAPE=4.27%

[INFO] Iteration 443: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 3.28%
[INFO] Predicted last sample of window: Actual=503.70, Pred=487.20, MAPE=3.28%

[INFO] Iteration 444: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.42%
[INFO] Predicted last sample of window: Actual=506.53, Pred=504.43, MAPE=0.42%

[INFO] Iteration 445: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.68%
[INFO] Predicted last sample of window: Actual=510.93, Pred=507.48, MAPE=0.68%

[INFO] Iteration 446: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.62%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 0.45%
[INFO] Predicted last sample of window: Actual=538.41, Pred=540.81, MAPE=0.45%

[INFO] Iteration 486: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.69%
[INFO] Predicted last sample of window: Actual=536.29, Pred=540.01, MAPE=0.69%

[INFO] Iteration 487: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.28%
[INFO] Predicted last sample of window: Actual=539.59, Pred=538.07, MAPE=0.28%

[INFO] Iteration 488: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.28%
[INFO] Predicted last sample of window: Actual=547.10, Pred=540.12, MAPE=1.28%

[INFO] Iteration 489: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.63%
[INFO] Predicted last sample of window: Actual=540.53, Pred=543.91, MAPE=0.63%

[INFO] Iteration 490: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.95%
[INFO] Predicted last samp